This NoteBook is used to cluster users and trying to give one direction of 
 venues recomendation based on response from clustering alg.


In [386]:
#Importing some libs
from pandas.io.json import json_normalize
import folium
from geopy.geocoders import Nominatim 
import requests
import pyfoursquare as foursquare
import pandas as pd
import statistics
from sklearn.cluster import KMeans

In [3]:
#Seting variables for auth on Foursquare API
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20190807'

In [50]:
def colect_venues(latitude,longitude,limite,raio):
    '''This function uses lat,long and limits to get venues in this area
       In the http query category is set to restaurantes, but we can set any catg'''
    neighborhood_latitude = latitude
    neighborhood_longitude = longitude
    LIMIT = limite
    radius = raio
    venues_in_range = 'https://api.foursquare.com/v2/venues/search?categoryId=4d4b7105d754a06374d81259&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
    request_response = requests.get(venues_in_range).json()
    venues = request_response['response']['venues']
    return venues

In [ ]:
def get_category_type(row):
    '''This funcion is used do clean up the categories response'''
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [51]:
#Get venues on area based on colect_venues function
possible_venues = colect_venues('40.758896','-73.985130',500,90000)

In [390]:
possible_venues

[{u'categories': [{u'icon': {u'prefix': u'https://ss3.4sqi.net/img/categories_v2/food/default_',
     u'suffix': u'.png'},
    u'id': u'4bf58dd8d48988d14e941735',
    u'name': u'American Restaurant',
    u'pluralName': u'American Restaurants',
    u'primary': True,
    u'shortName': u'American'}],
  u'hasPerk': False,
  u'id': u'462a6065f964a520d9451fe3',
  u'location': {u'address': u'1515 Broadway',
   u'cc': u'US',
   u'city': u'New York',
   u'country': u'United States',
   u'crossStreet': u'at W 45th St',
   u'distance': 131,
   u'formattedAddress': [u'1515 Broadway (at W 45th St)',
    u'New York, NY 10036',
    u'United States'],
   u'labeledLatLngs': [{u'label': u'display',
     u'lat': 40.758349343546215,
     u'lng': -73.98651265011574}],
   u'lat': 40.758349343546215,
   u'lng': -73.98651265011574,
   u'neighborhood': u'Theater District',
   u'postalCode': u'10036',
   u'state': u'NY'},
  u'name': u"Junior's Restaurant & Bakery",
  u'referralId': u'v-1567175362',
  u'venuePag

In [392]:
#Transforming Json on pandas DataFrane and clean some data
nearby_venues = json_normalize(possible_venues)
filtered_columns = ['id', 'name', 'categories', 'location.address', 'location.lat', 'location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]
nearby_venues['categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues['Likes'] = 'TBD'
nearby_venues['UserLikeList'] = 'TBD'

In [417]:
nearby_venues.head(18)

,id,name,categories,address,lat,lng,Likes,UserLikeList
0,462a6065f964a520d9451fe3,Junior's Restaurant & Bakery,American Restaurant,1515 Broadway,40.758349,-73.986513,TBD,TBD
1,428a8580f964a52083231fe3,Hard Rock Cafe,Theme Restaurant,1501 Broadway,40.757035,-73.986611,TBD,TBD
2,5a8c60ef91eaca3c3c812ce6,Chick-fil-A,Fast Food Restaurant,50 E 42nd St,40.752589,-73.979205,TBD,TBD
3,4e6e89b5d22daab469b62696,Catch,Seafood Restaurant,21 9th Ave,40.740127,-74.006202,TBD,TBD
4,5cd0190f3af988002c46b0c0,Essex Market,Food Court,88 Essex St,40.718025,-73.988247,TBD,TBD
5,579c12cd498e6e815b3acf50,Eataly Downtown,Market,101 Liberty St,40.710075,-74.011976,TBD,TBD
6,598de3c33b83073f81718721,Joe's Pizza,Pizza Place,1435 Broadway,40.754679,-73.987029,TBD,TBD
7,49f85cbbf964a520f26c1fe3,Mitsuwa Marketplace,Supermarket,595 River Rd,40.816251,-73.979988,TBD,TBD
8,5748d802498eccc4228c7fd1,City Vineyard,Wine Bar,233 West St,40.721193,-74.012921,TBD,TBD
9,594277b586f4cc0f251fc389,DeKalb Market Hall,Food Court,445 Albee Sq,40.691250,-73.982579,TBD,TBD


In [422]:
#Creating a folium instance map center on lat long passed before
Venues_Map = folium.Map(location=[40.758896,-73.985130],zoom_start=12)

#adding a circule on folium map center on lat long passed
folium.CircleMarker([40.758896,-73.985130],radius=10,popup='Ecco',fill=True,color='red',fill_color='red',fill_opacity=0.6).add_to(Venues_Map)


# add the trending venues as blue circle markers
for lat, lng, label in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name']):
    folium.CircleMarker([lat, lng],radius=5,poup=label,fill=True,color='blue',fill_color='blue',fill_opacity=0.6).add_to(Venues_Map)

In [423]:
Venues_Map

In [418]:
#getting user likes for all venues collected
#Whith this step we have one correlation with venues x users likes
Users = []
for index, row in nearby_venues.iterrows():
    like_list = []
    venues_likes = 'https://api.foursquare.com/v2/venues/{}/likes?&client_id={}&client_secret={}&v={}'.format(
    row['id'],
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    )
    likes = requests.get(venues_likes).json()
    for info in likes['response']['likes']['items']:
        like_list.append(info['id'])
        Users.append(info['id'])
    nearby_venues.at[index, 'UserLikeList'] = like_list
    nearby_venues.at[index, 'Likes'] = likes['response']['likes']['count']
   

In [419]:
nearby_venues

,id,name,categories,address,lat,lng,Likes,UserLikeList
0,462a6065f964a520d9451fe3,Junior's Restaurant & Bakery,American Restaurant,1515 Broadway,40.758349,-73.986513,1788,"[368000, 16189616, 453809652]"
1,428a8580f964a52083231fe3,Hard Rock Cafe,Theme Restaurant,1501 Broadway,40.757035,-73.986611,2951,"[409389877, 102083859, 551882420]"
2,5a8c60ef91eaca3c3c812ce6,Chick-fil-A,Fast Food Restaurant,50 E 42nd St,40.752589,-73.979205,41,"[146073114, 4160, 400223414]"
3,4e6e89b5d22daab469b62696,Catch,Seafood Restaurant,21 9th Ave,40.740127,-74.006202,1104,"[511343881, 394347798]"
4,5cd0190f3af988002c46b0c0,Essex Market,Food Court,88 Essex St,40.718025,-73.988247,54,"[3084086, 410193106, 517314948]"
5,579c12cd498e6e815b3acf50,Eataly Downtown,Market,101 Liberty St,40.710075,-74.011976,1143,"[152489174, 2332145, 58561787]"
6,598de3c33b83073f81718721,Joe's Pizza,Pizza Place,1435 Broadway,40.754679,-73.987029,289,"[349949235, 6228033, 226293226]"
7,49f85cbbf964a520f26c1fe3,Mitsuwa Marketplace,Supermarket,595 River Rd,40.816251,-73.979988,1252,"[13839018, 494421730]"
8,5748d802498eccc4228c7fd1,City Vineyard,Wine Bar,233 West St,40.721193,-74.012921,319,"[5906009, 7802215, 200826104]"
9,594277b586f4cc0f251fc389,DeKalb Market Hall,Food Court,445 Albee Sq,40.691250,-73.982579,743,"[417427355, 392376955]"


In [424]:
#Getting details for all users
Columns = ['ID','checkins','type','superuser','gender','homecity','n_photos','tips','mayorships','n_friends']

UsersDF = pd.DataFrame(columns = Columns)

for user in Users:
    inner_user_dict = {}
    url_users = 'https://api.foursquare.com/v2/users/{}?&client_id={}&client_secret={}&v={}'.format(
    user,
    CLIENT_ID, 
    CLIENT_SECRET,
    VERSION)
    results_users = requests.get(url_users).json()
    inner_user_dict['ID'] = results_users['response']['user']['id']
    inner_user_dict['checkins'] = results_users['response']['user']['checkins']['count']
    inner_user_dict['type'] = results_users['response']['user']['type']
    if 'superuser' in results_users['response']['user']:
        inner_user_dict['superuser'] = results_users['response']['user']['superuser']
    else:
        inner_user_dict['superuser'] = 0
    inner_user_dict['gender'] = results_users['response']['user']['gender']
    inner_user_dict['homecity'] = results_users['response']['user']['homeCity']
    inner_user_dict['n_photos'] = results_users['response']['user']['photos']['count']
    inner_user_dict['tips'] = results_users['response']['user']['tips']['count']
    inner_user_dict['mayorships'] = results_users['response']['user']['mayorships']['count']
    if 'friends' in results_users['response']['user']:
        inner_user_dict['n_friends'] = results_users['response']['user']['friends']['count']
    else:
        inner_user_dict['n_friends'] = 0
    inner_users_df = pd.DataFrame(inner_user_dict, index = ['ID'])
    UsersDF = UsersDF.append(inner_users_df, ignore_index = True)
    

In [426]:
UsersDF.head

<bound method DataFrame.head of             ID checkins  gender                             homecity  \
0       368000    10201  female                             New York   
1     16189616      968  female                        Cotia, Brasil   
2    453809652        1    male                                        
3    409389877      466  female                                        
4    102083859       19    male                          Ufa, Russia   
5    551882420       14    none                          Chicago, IL   
6    146073114      683    male                   New York, New York   
7         4160     4447    male               Poughkeepsie, New York   
8    400223414      155    none                         New York, NY   
9    511343881      726  female                                        
10   394347798      664  female                                        
11     3084086     5807    male                       Parsippany, NJ   
12   410193106      381  female 

In [427]:
#Replacing strings values for int values
gender_dict = {'female':1,'male':2,'none':3}
type_dict = {'user':1}
UsersDF = UsersDF.replace({'type': type_dict})
UsersDF = UsersDF.replace({'gender': gender_dict})
Homes_list = set(UsersDF['homecity'].tolist())

i = 0
home_location_dict = {}
for info in Homes_list:
    home_location_dict[info] =  i 
    i = i+1
    
UsersDF = UsersDF.replace({'homecity': home_location_dict})

In [429]:
UsersDF.head

<bound method DataFrame.head of             ID checkins  gender  homecity mayorships n_friends n_photos  \
0       368000    10201       1        59          0        48        0   
1     16189616      968       1        17          0        41        0   
2    453809652        1       2         0          0         0        0   
3    409389877      466       1         0          0         0        0   
4    102083859       19       2        14          0         0        0   
5    551882420       14       3        55          0         0        0   
6    146073114      683       2        12          0        16        0   
7         4160     4447       2        40          0        61        0   
8    400223414      155       3         8          0         0        0   
9    511343881      726       1         0          0         0        0   
10   394347798      664       1         0          0         0        0   
11     3084086     5807       2        15          0        20      

In [300]:
#Now we can use KMeans to clusters uses 
#Here we cluester users on 4 clusteres
UsersDF2 = UsersDF.drop(columns=['ID'])
km = KMeans(n_clusters = 4)
km.fit(UsersDF2.values)
prediction = km.predict(UsersDF2.values)
UsersDF['cluster'] = prediction

In [309]:
UsersDF

,ID,checkins,gender,homecity,mayorships,n_friends,n_photos,superuser,tips,type,cluster
0,368000,10185,1,56,0,48,0,1,52,1,2
1,16189616,963,1,19,0,41,0,0,22,1,1
2,453809652,1,2,0,0,0,0,0,0,1,1
3,551882420,14,3,52,0,0,0,0,0,1,1
4,152489174,2172,2,55,0,52,0,0,36,1,3
5,9014241,4460,1,18,0,0,0,0,1,1,3
6,146073114,683,2,14,0,16,0,0,1,1,1
7,4160,4444,2,39,0,61,0,0,3,1,3
8,400223414,155,3,10,0,0,0,0,48,1,1
9,511343881,720,1,0,0,0,0,0,1,1,1


In [374]:
#With each user in one cluester, we can add each venue like by user to user cluster
user_group = {}

for index, row in UsersDF.iterrows():
    if row['cluster'] in user_group:
        user_group[row['cluster']].append(row['ID'])
    else:
        user_group[row['cluster']] = [row['ID']]

In [379]:
#Whith each cluster user with all venues liked by uses on same cluester 
#We can adviser the most probabily venue for user on same cluster.
venues_vs_user_group = {}
for key,value in user_group.items():
    group_nearby_venues_list = []
    #print('Group',key,'Users:',value)
    for users_id in value:
        #print('testing user',users_id)
        for index, row in nearby_venues.iterrows():
            #print('testing row',index)
            like_list_to_test = row['UserLikeList']
            if users_id in like_list_to_test:
                group_nearby_venues_list.append(row['id'])
                #print('Matching for:',row['id'])
             
    #print('-->',group_nearby_venues_list,len(group_nearby_venues_list),len(set(group_nearby_venues_list)))
    venues_vs_user_group[key] = {'user_count':len(value),'users':value,'venues_count':len(set(group_nearby_venues_list)),
                                 'top_venue':  max(set(group_nearby_venues_list), key = group_nearby_venues_list.count) }
    print(venues_vs_user_group[key])
    print('------------------------------------------------------------------------------------')

{'venues_count': 9, 'user_count': 9, 'top_venue': u'5362a2ae498e3b18c22334be', 'users': [u'65890', u'13839018', u'3582397', u'7444795', u'2006845', u'71206817', u'1491653', u'367159', u'7447793']}
------------------------------------------------------------------------------------
{'venues_count': 44, 'user_count': 71, 'top_venue': u'5467ceec498e1b95c0575e73', 'users': [u'16189616', u'453809652', u'551882420', u'146073114', u'400223414', u'511343881', u'394347798', u'410193106', u'517314948', u'226293226', u'501320653', u'494421730', u'200826104', u'392376955', u'93943139', u'421280744', u'82677334', u'63996744', u'522280923', u'492037624', u'384468769', u'525854205', u'50201177', u'506876345', u'22691', u'60484468', u'505590360', u'487871986', u'49496611', u'460948204', u'479794342', u'17859217', u'35040924', u'533947886', u'45144401', u'3484561', u'34858101', u'498921705', u'4868604', u'82677334', u'19032000', u'118871833', u'555211531', u'441712', u'492037624', u'126784566', u'51411